In [397]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures

In [398]:
def removeValue(givenlist, valuelist):
    # Traverse the value list
    for value in valuelist:
       # using while loop to remove all occurrences of given value
        while(value in givenlist):
            # removing the value using remove()
            givenlist.remove(value)
           # return the list
    return givenlist

In [402]:
df = pd.read_csv("data/boligsiden_2.csv")

df.drop("Unnamed: 0", axis = 1, inplace = True)

df.drop("Unnamed: 0.1", axis = 1, inplace = True)

df

In [408]:
drop_list = ["caseUrl", "latitude", "longitude", "zipCode", "municipality", "perAreaPrice", "priceChangePercentage"]

X = df.drop(drop_list, axis = 1)

X.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

X = X.reset_index(drop=True)

#X

address_dummies = pd.get_dummies(X["addressType"])
energy_dummies = pd.get_dummies(X["energyLabel"])
municipality_dummies =  pd.get_dummies(X["municipalityCode"])

In [409]:
y = X["priceCash"]

In [410]:
X = X.drop("priceCash", axis = 1)

In [414]:
X_variables = X.columns.values.tolist()

X_variables = removeValue(X_variables, ["addressType", "energyLabel", "municipalityCode"])

X_variables

drop_list_2 = ["addressType", "energyLabel", "municipalityCode"]

X = X.drop(drop_list_2, axis = 1)

In [415]:
#X = pd.concat([X, pd.get_dummies(X["addressType"]), pd.get_dummies(X["energyLabel"]), pd.get_dummies(X["municipalityCode"])], axis=1)

In [416]:
X.numberOfBuildings.unique()

array([1], dtype=int64)

In [417]:
X = X.drop("numberOfBuildings", axis = 1) # drop number of buildings - those with more than one building (mostly farms) had NAN values for some variables

Dataframe X and DataFrames with Dummies

X

address_dummies

energy_dummies

municipality_dummies

In [418]:
X.columns = X.columns.astype(str)
address_dummies.columns = address_dummies.columns.astype(str)
energy_dummies.columns = energy_dummies.columns.astype(str)
municipality_dummies.columns = municipality_dummies.columns.astype(str)

In [420]:
address_dummies["basement"] = X.basement
address_dummies["lot"] = X.lot

X = X.drop(["basement", "lot"], axis = 1)

## 1. Only variables regarding the House itself

['basementArea',
 'housingArea',
 'lotArea',
 'monthlyExpense',
 'numberOfFloors',
 'numberOfRooms',
 'yearBuilt',
 'basement',
 'lot']
 
 

In [421]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [422]:
#X

In [423]:
poly = PolynomialFeatures(2, include_bias = False)
X_poly_int = poly.fit_transform(X)

In [424]:
X_poly_int = pd.DataFrame(X_poly_int, columns = poly.get_feature_names_out())

In [425]:
X_train, X_test, y_train, y_test = train_test_split(X_poly_int, y, test_size=.2, random_state=1)

"""pipe = make_pipeline(#PolynomialFeatures(interaction_only=True),
                    StandardScaler(),
                    LassoCV(max_iter = 10000))"""
scaler = StandardScaler()

X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.fit_transform(X_test)

# X_train_sc2 = scaler.transform(X_train_sc)
# X_test_sc2 = scaler.transform(X_test_sc)


reg = LassoCV(cv=5, random_state=0, max_iter = 10000).fit(X_train_sc, y_train)


y_pred = reg.predict(X_test_sc)

rmse_1 = np.sqrt(mse(y_pred, y_test))
score_1 = reg.score(X_train_sc, y_train)

In [426]:
rmse_1

1747939.2683861977

In [427]:
score_1

0.69747483140947

## address type dummies

In [428]:
address_dummies

,condo,cooperative,farm,full year plot,hobby farm,holiday house,terraced house,villa,villa apartment,basement,lot
0,0,0,0,0,0,0,0,1,0,1,1
1,0,0,1,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,1,0,1,1
3,0,0,0,0,0,0,0,1,0,1,1
4,0,0,0,0,0,0,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
8670,0,0,0,0,0,0,1,0,0,0,1
8671,0,0,0,0,0,0,0,1,0,0,1
8672,0,0,0,0,0,0,0,1,0,0,1
8673,0,0,0,0,0,0,0,1,0,0,1


In [429]:
X_poly_int_address = pd.concat([X_poly_int, address_dummies], axis=1)

In [430]:
#poly = PolynomialFeatures(interaction_only=True, include_bias = False)
#X = poly.fit_transform(X)

#X = pd.DataFrame(X, columns = poly.get_feature_names_out())

#X = X.loc[:, (X != 0).any(axis=0)]

#X = X.loc[:,~X.apply(lambda x: x.duplicated(),axis=1).all()].copy() # this removes the duplicates of interaction terms

In [431]:
X_train, X_test, y_train, y_test = train_test_split(X_poly_int_address, y, test_size=.2, random_state=1)
scaler = StandardScaler()

X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.fit_transform(X_test)

# X_train_sc2 = scaler.transform(X_train_sc)
# X_test_sc2 = scaler.transform(X_test_sc)


reg = LassoCV(cv=5, random_state=0, max_iter = 10000).fit(X_train_sc, y_train)


y_pred = reg.predict(X_test_sc)

rmse_2 = np.sqrt(mse(y_pred, y_test))
score_2 = reg.score(X_train_sc, y_train)

In [432]:
rmse_2

1677438.5819331682

In [433]:
score_2

0.7372850517407253

## Now also with energy codes

In [434]:
X_poly_int_address_energy = pd.concat([X_poly_int_address, energy_dummies], axis=1)

In [435]:
X_train, X_test, y_train, y_test = train_test_split(X_poly_int_address_energy, y, test_size=.2, random_state=1)
scaler = StandardScaler()

X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.fit_transform(X_test)

# X_train_sc2 = scaler.transform(X_train_sc)
# X_test_sc2 = scaler.transform(X_test_sc)


reg = LassoCV(cv=5, random_state=0, max_iter = 10000).fit(X_train_sc, y_train)


y_pred = reg.predict(X_test_sc)

rmse_3 = np.sqrt(mse(y_pred, y_test))
score_3 = reg.score(X_train_sc, y_train)

In [436]:
rmse_3

1659162.5717530884

In [437]:
score_3

0.744785439612457

## Now with house variables, address type, energy and municipality


In [438]:
X_poly_int_address_energy_municipality = pd.concat([X_poly_int_address_energy, municipality_dummies], axis=1)

In [439]:
X_train, X_test, y_train, y_test = train_test_split(X_poly_int_address_energy_municipality, y, test_size=.2, random_state=1)
scaler = StandardScaler()

X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.fit_transform(X_test)

# X_train_sc2 = scaler.transform(X_train_sc)
# X_test_sc2 = scaler.transform(X_test_sc)


reg = LassoCV(cv=5, random_state=0, max_iter = 10000).fit(X_train_sc, y_train)


y_pred = reg.predict(X_test_sc)

rmse_4 = np.sqrt(mse(y_pred, y_test))
score_4 = reg.score(X_train_sc, y_train)

In [440]:
rmse_4

1504671.4533923788

In [441]:
score_4

0.8039550048457592

In [442]:
#.coef_

In [461]:
address_dummies = pd.get_dummies(X["addressType"])
energy_dummies = pd.get_dummies(X["energyLabel"])
municipality_dummies =  pd.get_dummies(X["municipalityCode"])

KeyError: 'addressType'

In [453]:
X_list = [X_poly_int, X_poly_int_address, X_poly_int_address_energy, X_poly_int_address_energy_municipality]

In [458]:
reg_scores = []
reg_rmses = []
reg_coefficients = []
for X in X_list:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)
    scaler = StandardScaler()

    X_train_sc = scaler.fit_transform(X_train)
    X_test_sc = scaler.fit_transform(X_test)


    reg = LassoCV(cv=5, random_state=0, max_iter = 10000).fit(X_train_sc, y_train)


    y_pred = reg.predict(X_test_sc)

    reg_rmses.append(np.sqrt(mse(y_pred, y_test)))
    reg_scores.append(reg.score(X_train_sc, y_train))
    reg_coefficients.append(reg.coef_)

In [459]:
reg_rmses

[1747939.2683861977,
 1677438.5819331682,
 1659162.5717530884,
 1504671.4533923788]

In [460]:
reg_scores

[0.69747483140947, 0.7372850517407253, 0.744785439612457, 0.8039550048457592]